TUBES visdat

-Yudhistira Imam praja - 1301204192

-Dwina Sarah Delva - 1301204124


In [67]:
!pip install bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
import pandas as pd
from bokeh.io import curdoc
from bokeh.plotting import figure, output_file,show
from bokeh.io import curdoc, show, output_notebook, output_file
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.layouts import widgetbox, row
from bokeh.models import Select, RangeSlider
from datetime import date, datetime
output_notebook()

In [69]:
cov_data = pd.read_csv('covid_19_indonesia_time_series_all.csv')
cov_data.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Total Deaths per 100rb,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,39,20,75,...,-6.204699,0.18,3.60,0.0,1.84,0.18,51.28%,192.31%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,41,20,75,...,-6.204699,0.18,3.78,0.0,1.84,0.18,48.78%,182.93%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,-0.789275,0.01,0.01,0.0,0.00,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-RI,Riau,1,0,0,1,1,0,1,...,0.511648,0.16,0.16,0.0,0.00,0.00,0.00%,100.00%,NaN,NaN
4,3/3/2020,ID-JK,DKI Jakarta,2,0,0,2,43,20,75,...,-6.204699,0.18,3.96,0.0,1.84,0.18,46.51%,174.42%,1.0,1.0


** Preprocessing Data**

In [70]:
# Selecting column usage
df = cov_data.iloc[:,:12]

# remove data with location level = Country
idx = df[df['Location Level'] == 'Country'].index
df.drop(idx, axis=0, inplace=True)


In [71]:
# remove unused columns
col = ['Location ISO Code', 'New Cases', 'New Deaths', 'New Recovered', 'New Active Cases', 'Location Level']
df.drop(col, axis=1, inplace=True)

# rename columns
col = {
    'Total Cases': 'Total_Cases',
    'Total Deaths': 'Total_Deaths',
    'Total Recovered': 'Total_Recovered',
    'Total Active Cases': 'Total_Active_Cases'}
df.rename(col, axis=1, inplace=True)



# convert string to datetime for column Date
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# make a new column for month and year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['year'] = pd.DatetimeIndex(df['Date']).year


Bokeh Data Visualization

In [72]:
# Make a list of the unique values from index (provinsi)
prov_list = list(df.Location.unique())

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=prov_list)

#Change datatype year from int to str
df['year'] = df['year'].apply(str)


# Make the ColumnDataSource: source
source = ColumnDataSource(data={
    'x'       : df['Date'].loc[(df.Location == 'DKI Jakarta') & (df.month.isin([i for i in range(3,6)])) & (df.year == '2020')],
    'y'       : df['Total_Deaths'].loc[(df.Location == 'DKI Jakarta') & (df.month.isin([i for i in range(3,6)])) & (df.year == '2020')]
    })


**updating data**

In [73]:

#Callback for updating data
def callback(attr, old, new):
    minMonth, maxMonth = slider.value
    selectedLocation = loc_select.value
    selectedData = data_select.value
    selectedYear = yr_select.value

    # Label axes of plot
    plot.yaxis.axis_label = selectedData

    # new data
    new_data = {
        'x'       : df['Date'].loc[(df.Location == selectedLocation) & (df.month.isin([i for i in range(int(minMonth),int(maxMonth+1))])) & (df.year == selectedYear)],
        'y'       : df[selectedData].loc[(df.Location == selectedLocation) & (df.month.isin([i for i in range(int(minMonth),int(maxMonth+1))])) & (df.year == selectedYear)]
    }
    # updating source data to new data
    source.data = new_data

Plotting

In [74]:
#Define figure usage
plot = figure(title='Visualisasi Covid-19 per Provinsi', x_axis_label='Date', x_axis_type="datetime", y_axis_label='Total_Deaths',
           plot_height=400, plot_width=700)

# plotting for line chart
plot.line(x='x', y='y', source=source)

#adding tools to chart
plot.add_tools(HoverTool(
    tooltips=[
        ( 'date','@x{%F}'),
        ( 'Value', '@y'),
    ],

    formatters={
        '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
))


Bokeh widget

In [75]:
#RangeSlider for selectedMonth
slider = RangeSlider(start=1, end=12, step=1, value=(3,5), title='Bulan')
slider.on_change('value',callback)


In [76]:
#Dropdown for selectedLocation
loc_select = Select(
    options= list(df.Location.unique()),
    value='DKI Jakarta',
    title='Provinsi'
)
loc_select.on_change('value', callback)

#Dropdown for selectedYear
yr_select = Select(
    options= ['2020','2021'],
    value='2020',
    title='Tahun'
)
yr_select.on_change('value', callback)

#Dropdown for selectedData
data_select = Select(
    options= ["Total_Cases","Total_Deaths","Total_Recovered","Total_Active_Cases"],
    value='Total_Deaths',
    title='Data Covid-19'
)
data_select.on_change('value', callback)


In [77]:
# Creating layout
layout = row(widgetbox(yr_select,slider,loc_select,data_select), plot)
curdoc().add_root(layout)
curdoc().title = 'Visualisasi Covid-19 per Provinsi'
output_file('result.html')
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

